##Realizando extração e transformação inicial

In [0]:
# Importar bibliotecas do PySpark

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# Utilizando biblioteca DBUtils (nativa) para apontar caminho de onde os arquivos serão importados # dbfs://FileStore

# Limpar a pasta tables
#dbutils.fs.rm('dbfs:/FileStore/tables', True)

# Armazenar em variavel a lista das infos dos arquivos no caminho especificado
files = dbutils.fs.ls("dbfs:/FileStore/tables")
files


[FileInfo(path='dbfs:/FileStore/tables/Sales_April_2019.csv', name='Sales_April_2019.csv', size=1595953, modificationTime=1717289309000),
 FileInfo(path='dbfs:/FileStore/tables/Sales_August_2019.csv', name='Sales_August_2019.csv', size=1043593, modificationTime=1717289309000),
 FileInfo(path='dbfs:/FileStore/tables/Sales_December_2019.csv', name='Sales_December_2019.csv', size=2181642, modificationTime=1717289311000),
 FileInfo(path='dbfs:/FileStore/tables/Sales_February_2019.csv', name='Sales_February_2019.csv', size=1046495, modificationTime=1717289310000),
 FileInfo(path='dbfs:/FileStore/tables/Sales_January_2019.csv', name='Sales_January_2019.csv', size=843098, modificationTime=1717289311000),
 FileInfo(path='dbfs:/FileStore/tables/Sales_January_2020.csv', name='Sales_January_2020.csv', size=1595953, modificationTime=1717289312000),
 FileInfo(path='dbfs:/FileStore/tables/Sales_July_2019.csv', name='Sales_July_2019.csv', size=1248753, modificationTime=1717289312000),
 FileInfo(path=

In [0]:
# filtrar arquivos .csv apenas de 2019 usando apenas o path

files_2019 = [f.path for f in files if f.path.endswith("2019.csv")]
files_2019

['dbfs:/FileStore/tables/Sales_April_2019.csv',
 'dbfs:/FileStore/tables/Sales_August_2019.csv',
 'dbfs:/FileStore/tables/Sales_December_2019.csv',
 'dbfs:/FileStore/tables/Sales_February_2019.csv',
 'dbfs:/FileStore/tables/Sales_January_2019.csv',
 'dbfs:/FileStore/tables/Sales_July_2019.csv',
 'dbfs:/FileStore/tables/Sales_June_2019.csv',
 'dbfs:/FileStore/tables/Sales_March_2019.csv',
 'dbfs:/FileStore/tables/Sales_May_2019.csv',
 'dbfs:/FileStore/tables/Sales_November_2019.csv',
 'dbfs:/FileStore/tables/Sales_October_2019.csv',
 'dbfs:/FileStore/tables/Sales_September_2019.csv']

In [0]:
# Criar primeiro dataframe (inicial)

df = spark.read.csv(files_2019[0], inferSchema=True, header=True)
display(df.limit(10))

Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
null,null,null,null,null,null
176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
176560,Google Phone,1,600.0,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
176562,USB-C Charging Cable,1,11.95,04/29/19 13:03,"381 Wilson St, San Francisco, CA 94016"
176563,Bose SoundSport Headphones,1,99.99,04/02/19 07:46,"668 Center St, Seattle, WA 98101"
176564,USB-C Charging Cable,1,11.95,04/12/19 10:58,"790 Ridge St, Atlanta, GA 30301"
176565,Macbook Pro Laptop,1,1700.0,04/24/19 10:38,"915 Willow St, San Francisco, CA 94016"


In [0]:
# Criar os demais DFs para cada mês (a partir do segundo arquivo) e juntar (union) com o primeiro DF

for f in files_2019[1:]:
    df_temp = spark.read.csv(f, inferSchema=True,header=True)
    df = df.union(df_temp)
    print(f"{f} incluído com sucesso!")

dbfs:/FileStore/tables/Sales_August_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_December_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_February_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_January_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_July_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_June_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_March_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_May_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_November_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_October_2019.csv incluído com sucesso!
dbfs:/FileStore/tables/Sales_September_2019.csv incluído com sucesso!


In [0]:
# Verificar se outros meses foram importados além de Abril 

display(df.filter(col("Order Date").startswith("08")).limit(10))

Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
236670,Wired Headphones,2,11.99,08/31/19 22:21,"359 Spruce St, Seattle, WA 98101"
236671,Bose SoundSport Headphones,1,99.99,08/15/19 15:11,"492 Ridge St, Dallas, TX 75001"
236672,iPhone,1,700.0,08/06/19 14:40,"149 7th St, Portland, OR 97035"
236673,AA Batteries (4-pack),2,3.84,08/29/19 20:59,"631 2nd St, Los Angeles, CA 90001"
236674,AA Batteries (4-pack),2,3.84,08/15/19 19:53,"736 14th St, New York City, NY 10001"
236675,Wired Headphones,1,11.99,08/02/19 23:54,"470 Hill St, San Francisco, CA 94016"
236676,34in Ultrawide Monitor,1,379.99,08/04/19 19:52,"470 Cherry St, Los Angeles, CA 90001"
236677,20in Monitor,1,109.99,08/13/19 07:16,"918 6th St, San Francisco, CA 94016"
236678,Wired Headphones,1,11.99,08/25/19 20:11,"58 9th St, San Francisco, CA 94016"
236679,Macbook Pro Laptop,1,1700.0,08/07/19 15:43,"239 Spruce St, Los Angeles, CA 90001"


In [0]:
# Criar novas colunas no DataFrame para eventuais análises

# Colunas para determinar mês, dia e hora
df = df.withColumn("Month",substring(col("Order Date"),0,2))\
    .withColumn("Day",substring(col("Order Date"),4,2))\
    .withColumn("Time",substring(col("Order Date"),10,5))

display(df.limit(10))

Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Day,Time
176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",04,19,08:46
null,null,null,null,null,null,null,null,null
176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",04,07,22:30
176560,Google Phone,1,600.0,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38
176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38
176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",04,30,09:27
176562,USB-C Charging Cable,1,11.95,04/29/19 13:03,"381 Wilson St, San Francisco, CA 94016",04,29,13:03
176563,Bose SoundSport Headphones,1,99.99,04/02/19 07:46,"668 Center St, Seattle, WA 98101",04,02,07:46
176564,USB-C Charging Cable,1,11.95,04/12/19 10:58,"790 Ridge St, Atlanta, GA 30301",04,12,10:58
176565,Macbook Pro Laptop,1,1700.0,04/24/19 10:38,"915 Willow St, San Francisco, CA 94016",04,24,10:38


In [0]:
# Verificando quantidade de valores nulos em cada coluna

for column in df.columns:
    print(column, df.filter(col(column).isNull()).count())

Order ID 900
Product 545
Quantity Ordered 900
Price Each 900
Order Date 545
Purchase Address 545
Month 545
Day 545
Time 545


In [0]:
# Remover duplicatas
df = df.dropDuplicates()

# Exibir valores nulos restantes
display(df.filter(col("Order ID").isNull()))

Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Day,Time
null,null,null,null,null,null,null,null,null
null,Product,null,null,Order Date,Purchase Address,Or,er,e


In [0]:
# Trazer apenas valores não nulos
df = df.filter(col("Order ID").isNotNull())
display(df.limit(10))


Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Day,Time
176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",04,30,09:27
176560,Google Phone,1,600.0,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38
176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38
176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",04,07,22:30
176562,USB-C Charging Cable,1,11.95,04/29/19 13:03,"381 Wilson St, San Francisco, CA 94016",04,29,13:03
176564,USB-C Charging Cable,1,11.95,04/12/19 10:58,"790 Ridge St, Atlanta, GA 30301",04,12,10:58
176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",04,19,08:46
176566,Wired Headphones,1,11.99,04/08/19 14:05,"83 7th St, Boston, MA 02215",04,08,14:05
176563,Bose SoundSport Headphones,1,99.99,04/02/19 07:46,"668 Center St, Seattle, WA 98101",04,02,07:46
176565,Macbook Pro Laptop,1,1700.0,04/24/19 10:38,"915 Willow St, San Francisco, CA 94016",04,24,10:38


In [0]:
# Remover coluna desnecessária (data) e alterar coluna de qtd pra numerica
df = df.drop(col("Order Date"))
display(df.limit(10))

Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Month,Day,Time
176561,Wired Headphones,1,11.99,"333 8th St, Los Angeles, CA 90001",04,30,09:27
176560,Google Phone,1,600.0,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38
176560,Wired Headphones,1,11.99,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38
176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",04,07,22:30
176562,USB-C Charging Cable,1,11.95,"381 Wilson St, San Francisco, CA 94016",04,29,13:03
176564,USB-C Charging Cable,1,11.95,"790 Ridge St, Atlanta, GA 30301",04,12,10:58
176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",04,19,08:46
176566,Wired Headphones,1,11.99,"83 7th St, Boston, MA 02215",04,08,14:05
176563,Bose SoundSport Headphones,1,99.99,"668 Center St, Seattle, WA 98101",04,02,07:46
176565,Macbook Pro Laptop,1,1700.0,"915 Willow St, San Francisco, CA 94016",04,24,10:38


In [0]:
# Criar nova coluna de valor total
df = df.withColumn("Total",col("Quantity Ordered")*col("Price Each"))
display(df.limit(10))

Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Month,Day,Time,Total
176561,Wired Headphones,1,11.99,"333 8th St, Los Angeles, CA 90001",04,30,09:27,11.99
176560,Google Phone,1,600.0,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38,600.0
176560,Wired Headphones,1,11.99,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38,11.99
176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",04,07,22:30,99.99
176562,USB-C Charging Cable,1,11.95,"381 Wilson St, San Francisco, CA 94016",04,29,13:03,11.95
176564,USB-C Charging Cable,1,11.95,"790 Ridge St, Atlanta, GA 30301",04,12,10:58,11.95
176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",04,19,08:46,23.9
176566,Wired Headphones,1,11.99,"83 7th St, Boston, MA 02215",04,08,14:05,11.99
176563,Bose SoundSport Headphones,1,99.99,"668 Center St, Seattle, WA 98101",04,02,07:46,99.99
176565,Macbook Pro Laptop,1,1700.0,"915 Willow St, San Francisco, CA 94016",04,24,10:38,1700.0


In [0]:
# Ajustar ordem das colunas
df.columns

df = df.select(['Order ID',
 'Product',
 'Quantity Ordered',
 'Price Each',
 'Total',
 'Purchase Address',
 'Month',
 'Day',
 'Time'])

display(df.limit(10))

Order ID,Product,Quantity Ordered,Price Each,Total,Purchase Address,Month,Day,Time
176561,Wired Headphones,1,11.99,11.99,"333 8th St, Los Angeles, CA 90001",04,30,09:27
176560,Google Phone,1,600.0,600.0,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38
176560,Wired Headphones,1,11.99,11.99,"669 Spruce St, Los Angeles, CA 90001",04,12,14:38
176559,Bose SoundSport Headphones,1,99.99,99.99,"682 Chestnut St, Boston, MA 02215",04,07,22:30
176562,USB-C Charging Cable,1,11.95,11.95,"381 Wilson St, San Francisco, CA 94016",04,29,13:03
176564,USB-C Charging Cable,1,11.95,11.95,"790 Ridge St, Atlanta, GA 30301",04,12,10:58
176558,USB-C Charging Cable,2,11.95,23.9,"917 1st St, Dallas, TX 75001",04,19,08:46
176566,Wired Headphones,1,11.99,11.99,"83 7th St, Boston, MA 02215",04,08,14:05
176563,Bose SoundSport Headphones,1,99.99,99.99,"668 Center St, Seattle, WA 98101",04,02,07:46
176565,Macbook Pro Laptop,1,1700.0,1700.0,"915 Willow St, San Francisco, CA 94016",04,24,10:38


#### Transformando dataframe tratado para responder perguntas de negócios

In [0]:
# Agregar valores totais por mês de vendas

df_total_per_month = df.groupBy("Month").agg({"Total":"sum"})
df_total_per_month = df_total_per_month.withColumnRenamed("sum(Total)","Total")\
    .withColumn("Total",round("Total",2))\
        .withColumn("Total",format_number(col("Total"),2))\
        .sort(col("Total"),ascending=False)
display(df_total_per_month)

Month,Total
12,"4,608,295.70"
10,"3,734,777.86"
04,"3,389,217.98"
11,"3,197,875.05"
05,"3,150,616.23"
03,"2,804,973.35"
07,"2,646,461.32"
06,"2,576,280.15"
08,"2,241,083.37"
02,"2,200,078.08"


In [0]:
# Qual cidade vendeu mais?

df_total_per_city = df.withColumn("AddressList",split(col("Purchase Address"),", "))\
    .withColumn("City",trim(col("AddressList")[1]))\
        .withColumn("State",trim(col("AddressList")[2]))\
        .withColumn("State",split(col("State")," ")[0])\
            .withColumn("City",concat_ws(" - ",col("City"),col("State")))\
            .drop("State","AddressList")

df_total_per_city = df_total_per_city.groupBy("City").agg({"Total":"sum"}).withColumnRenamed("sum(Total)","Total")\
    .withColumn("Total",round(col("Total"),2))\
        .withColumn("Total",format_number(col("Total"),2))\
        .sort(col("Total"),ascending=False)

display(df_total_per_city)

City,Total
San Francisco - CA,"8,254,743.55"
Los Angeles - CA,"5,448,304.28"
Portland - ME,"449,321.38"
New York City - NY,"4,661,867.14"
Boston - MA,"3,658,627.65"
Atlanta - GA,"2,794,199.07"
Dallas - TX,"2,765,373.96"
Seattle - WA,"2,745,046.02"
Portland - OR,"1,870,010.56"
Austin - TX,"1,818,044.33"
